# Laboratorio 11 - Data Science
## Proyecto de Consultoría Regresión Logística
- Nelson García Bravatti
- Christian Echeverría


In [ ]:
!pip -q install pyspark

In [ ]:
from pyspark.sql import SparkSession

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

spark = SparkSession.builder.appName("Lab DF — Abandono").getOrCreate()
print("Spark version:", spark.version)

In [ ]:
file_path = 'abandono_clientes.csv'

df = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(file_path)
)

# Ver las primeras filas del dataframe
df.show(5)

In [ ]:
df.printSchema()